# Twitter analysis for social unrest

In [6]:
import numpy as np
import pandas as pd

#Displaying data as dataframe
#pd.read_csv reads a comma-separated values (csv) file into dataframe.
df = pd.read_csv("data/dummydata.csv")

#Displaying certain columns from the dataframe to display - 
# df.loc accesses a group of rows and columns by label(s) from dataframe.
df = df.loc[:100,['text', 'retweets', 'query', 'user_location']]

#Output the dataframe 
df

,text,retweets,query,user_location
0,we can adjust by population to get crude exces...,108,death rates,NaN
1,turning to labour or remainers like what would...,116,death rates,NaN
2,numbers of deaths are affected by population s...,75,death rates,NaN
3,here is a list of governors who preside over s...,7612,death rates,NaN
4,convince me with facts on death rates and infe...,0,death rates,NaN
...,...,...,...,...
96,blatant lies inflated death rates keeping the ...,0,death rates,NaN
97,comparison with other countries isnt allowed w...,0,death rates,NaN
98,i didnt say it played no part but its pointles...,0,death rates,NaN
99,although why would you fly here with our world...,0,death rates,NaN
